In [41]:
import pandas as pd
import random
import simplekml
from fastkml import kml
from lxml import html
import pandas as pd
from zipfile import ZipFile
from pyproj import Transformer
from shapely.ops import transform
from shapely.geometry import Point
from pyproj import Geod
from shapely import LineString
from shapely.ops import substring
from shapely import MultiLineString
import warnings
import pandas as pd
import os
import re
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

from kmz_module import get_homepass_folder, get_placemark, to_df

def getAllHP(file_path):
    placemark_dict = get_placemark(file_path)
    hp_folder_name = get_homepass_folder(placemark_dict)
    all_homepass_df = pd.DataFrame()  # DataFrame kosong untuk menyimpan semua data

    for name in hp_folder_name:
        homepass_df = to_df(placemark_dict[name], parse_simple=True)
        all_homepass_df = pd.concat([all_homepass_df, homepass_df], ignore_index=True)  # Gabungkan DataFrame ke DataFrame besar

    return all_homepass_df

def getAllFAT(file_path):
    placemark_dict = get_placemark(file_path)
    fat_df = to_df(placemark_dict["FAT"], parse_simple=False)    

    return fat_df

In [7]:
import json
import requests
import time

while True:
    force_base_url = 'http://localhost:5000'

    url = f'{force_base_url}/get_path_cluster_id'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if not data:  # Cek jika data kosong
            print('No Documents to Check.')
            standby = True
        else:
            standby = False
            file_path = data.get('file_path')
            cluster_id = data.get('cluster_id')
            print(f'File Path: {file_path}, Cluster ID: {cluster_id}')
    else:
        print('Failed to get data. Status code:', response.status_code)
    
    time.sleep(2)

    url = f'{force_base_url}/update_processed'

    data = {
        'cluster_id': cluster_id
    }
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        print('Data inserted successfully.')
    else:
        print('Failed to insert data. Status code:', response.status_code)

    time.sleep(2)

File Path: D:\Documents\MBKM\force\Input\XL33BYM01511_001, Cluster ID: XL33BYM01511_001
Data inserted successfully.
File Path: D:\Documents\MBKM\force\Input\XL32IMY00164_002, Cluster ID: XL32IMY00164_002
Data inserted successfully.


ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /get_path_cluster_id (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002039D9675D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
fat_long_lat = getAllFAT('Input\ABD - XLSGM1218.kmz')
print(fat_long_lat)

                    Name              Coordinates
0  SGM1-JSA01-H12-D03-07  [119.445176, -5.219916]
1  SGM1-JSA01-H12-D03-06  [119.444914, -5.219481]
2  SGM1-JSA01-H12-D03-05  [119.443971, -5.219344]
3  SGM1-JSA01-H12-D03-08  [119.446108, -5.220035]
4  SGM1-JSA01-H12-D03-04  [119.442964, -5.219066]
5  SGM1-JSA01-H12-D03-09  [119.445209, -5.221219]


In [46]:
coordinate_col = ['BUILDING_LONGITUDE', 'BUILDING_LATITUDE']
homepass_long_lat = getAllHP('Input\ABD - XLSGM1218.kmz')
red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')

hpdb_df = pd.read_excel('Input\HPDB - XLSGM1218.xlsx', sheet_name='HPDB_Excel')
wb = load_workbook('Input\HPDB - XLSGM1218.xlsx')
ws = wb.active

for index, row in hpdb_df.iterrows():
        build_longitude = row['BUILDING_LONGITUDE']
        build_latitude = row['BUILDING_LATITUDE']
        fat_longitude = row['FAT_LONGITUDE']
        fat_latitude = row['FAT_LATITUDE']
        if pd.notnull(build_longitude) and pd.notnull(build_latitude):
            # Ubah nilai longitude dan latitude menjadi tuple
            coordinate_tuple = [float(build_longitude), float(build_latitude)]

            if not str(coordinate_tuple) in homepass_long_lat["Coordinates"].astype(str).to_list():
                # Tambahkan logika untuk menandai jika nilai tidak ditemukan
                ws.cell(row=index + 2, column=hpdb_df.columns.get_loc('BUILDING_LONGITUDE') + 1).fill = red_fill
                ws.cell(row=index + 2, column=hpdb_df.columns.get_loc('BUILDING_LATITUDE') + 1).fill = red_fill

        if pd.notnull(fat_longitude) and pd.notnull(fat_latitude):
            # Ubah nilai longitude dan latitude menjadi tuple
            coordinate_tuple = [float(fat_longitude), float(fat_latitude)]

            if not str(coordinate_tuple) in fat_long_lat["Coordinates"].astype(str).to_list():
                # Tambahkan logika untuk menandai jika nilai tidak ditemukan
                ws.cell(row=index + 2, column=hpdb_df.columns.get_loc('FAT_LONGITUDE') + 1).fill = red_fill
                ws.cell(row=index + 2, column=hpdb_df.columns.get_loc('FAT_LATITUDE') + 1).fill = red_fill

wb.save('Output\output_HPDB - XLSGM1218.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'Input\\ABD - XLSGM1218.kmz'

In [36]:
print(coordinate_tuple)
print(homepass_long_lat['Coordinates'][91])
print(str(coordinate_tuple) in homepass_long_lat["Coordinates"].astype(str).to_list())
# """for i in homepass_long_lat["Coordinates"]:
#     if i == coordinate_tuple:
#         print("ada yang sama bro")"""

[119.445787, -5.220373]
[119.445787, -5.220373]
True


In [15]:
print(homepass_long_lat['Coordinates'])

0     [119.445351, -5.219761]
1     [119.445168, -5.219991]
2     [119.445052, -5.219981]
3     [119.444979, -5.219891]
4     [119.444987, -5.219637]
               ...           
87    [119.444176, -5.219279]
88    [119.443645, -5.219123]
89    [119.445607, -5.220439]
90    [119.445661, -5.219887]
91    [119.445787, -5.220373]
Name: Coordinates, Length: 92, dtype: object


In [ ]:
import requests

# SharePoint site URL
site_url = 'https://xlo365-my.sharepoint.com/personal/mba_fajarp_xl_co_id/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fmba_fajarp_xl_co_id%2FDocuments%2FTES+FOLDER+X&ga=1'

# Path to the file you want to upload
file_path = '/path/to/your/file.txt'

# SharePoint document library where you want to upload the file
library_name = 'Documents'

# SharePoint API endpoint for uploading files
upload_url = f'{site_url}/_api/web/GetFolderByServerRelativeUrl(\'{library_name}\')/Files/add(url=\'{file_path}\', overwrite=true)'

# SharePoint credentials (username and password)
username = 'your_username'
password = 'your_password'

# Perform the upload request
with open(file_path, 'rb') as file:
    headers = {
        'Content-Type': 'application/json;odata=verbose',
        'Accept': 'application/json;odata=verbose',
        'X-RequestDigest': 'FormDigestValue',
    }
    response = requests.post(upload_url, auth=(username, password), headers=headers, data=file)

# Check if the upload was successful
if response.status_code == 200:
    print('File uploaded successfully')
else:
    print('Failed to upload file')
    print(response.text)


In [4]:
import datetime


today = datetime.date.today()
today_time = today.strftime('%d-%m-%Y')
yesterday = today - datetime.timedelta(days=1)
yesterday_time = yesterday.strftime('%d-%m-%Y')
print(yesterday_time)
print(today_time)

26-03-2024
27-03-2024
